# MSDM5004 Homework 2

## 1. Write a code using to compute the derivative of 
$f(x) = \frac{\sin x}{x^3}$ at $x = 4$ using the central difference formula, and check the order of
convergence.

In [1]:
function central_diff(f::Function,x::Real,h::Real)
    return (f(x+h)-f(x-h))/2/h
end

central_diff (generic function with 1 method)

In [2]:
f(x) = sin(x)/x^3

f (generic function with 1 method)

In [3]:
v = central_diff(f,4,1e-3)
display("text/markdown","The derivative of f(x) at x=4 is $v")

The derivative of f(x) at x=4 is -0.0013444070485774812

Then let check the order of convergence:

In [4]:
n = 10
h₀ = 0.1
hs = h₀ * (1/2) .^ collect(0:(n-1))
phi = central_diff.(f,4,hs)
rs = (phi[1:end-2]-phi[2:end-1])./(phi[2:end-1]-phi[3:end])
for r in rs
    @show r
end

r = 4.002842241732386
r = 4.000710087426402
r = 4.000177492575602
r = 4.000044360840964
r = 4.000011257409564
r = 4.000002797273308
r = 3.999998456677391
r = 3.9999876534572376


## 2. Find the first two iterations using 
- (1) the Jacobi method
- (2) the Gauss-Seidel method 

for the following linear system, starting from $x^{(0)} = (0, 0, -1)$.

$$
\begin{aligned}
4x_1 −x_2 + x_3 &=3,\\
2x_1 + 5x_2 + 2x_3 &=4,\\
−x_1 + x_2 + 3x_3 &= −6.
\end{aligned}
$$

In [5]:
using LinearAlgebra

include("../utils/MatrixDisplay.jl")

function DLU(A::Matrix{<:Real})
    D = Diagonal(A)
    L = D-LowerTriangular(A)
    U = D-UpperTriangular(A)
    return D,L,U
end

DLU (generic function with 1 method)

In [6]:
A = [
    4 -1 1;
    2 5 2;
    -1 1 3
];
b = [3;4;-6];
x⁽⁰⁾ = [0;0;-1];

First of all, let's decompose matrix A into D,L,U

In [7]:
D,L,U=DLU(A);

In [8]:
@latex D

<center>$D=\begin{pmatrix}4&0&0\\0&5&0\\0&0&3\end{pmatrix}$</center>


In [9]:
@latex L

<center>$L=\begin{pmatrix}0&0&0\\-2&0&0\\1&-1&0\end{pmatrix}$</center>


In [10]:
@latex U

<center>$U=\begin{pmatrix}0&1&-1\\0&0&-2\\0&0&0\end{pmatrix}$</center>


### (1) Jacobi method

In [11]:
@latex Tⱼ = D^(-1)*(L+U)

<center>$Tⱼ=\begin{pmatrix}0.0&0.25&-0.25\\-0.4&0.0&-0.4\\0.3333333333333333&-0.3333333333333333&0.0\end{pmatrix}$</center>


In [12]:
@latex cⱼ = D^(-1)*b

<center>$cⱼ=\begin{pmatrix}0.75\\0.8\\-2.0\end{pmatrix}$</center>


In [13]:
@latex x⁽¹⁾ = Tⱼ*x⁽⁰⁾+cⱼ

<center>$x⁽¹⁾=\begin{pmatrix}1.0\\1.2000000000000002\\-2.0\end{pmatrix}$</center>


In [14]:
@latex x⁽²⁾ = Tⱼ*x⁽¹⁾+cⱼ

<center>$x⁽²⁾=\begin{pmatrix}1.55\\1.2000000000000002\\-2.066666666666667\end{pmatrix}$</center>


### (2) Gauss-Seidel method

In [15]:
@latex Tg = (D-L)^(-1)*U

<center>$Tg=\begin{pmatrix}0.0&0.25&-0.25\\0.0&-0.1&-0.30000000000000004\\0.0&0.11666666666666665&0.016666666666666677\end{pmatrix}$</center>


In [16]:
@latex cg = (D-L)^(-1)*b

<center>$cg=\begin{pmatrix}0.75\\0.5\\-1.9166666666666667\end{pmatrix}$</center>


In [17]:
@latex x⁽¹⁾=Tg*x⁽⁰⁾+cg

<center>$x⁽¹⁾=\begin{pmatrix}1.0\\0.8\\-1.9333333333333333\end{pmatrix}$</center>


In [18]:
@latex x⁽²⁾=Tg*x⁽¹⁾+cg

<center>$x⁽²⁾=\begin{pmatrix}1.4333333333333333\\1.0\\-1.8555555555555556\end{pmatrix}$</center>


## 3. Solve the following linear system using 
- (1) the Gauss-Seidel method,
- (2) the SOR method with ω =1.2. 

The initial estimate is $x^{(0)} = (0, 0, 1, 0)$. 

Stop the iterations until the $l_{\infty}$ norm
$||x^{(k)}-x^{(k-1)}||_{\infty} \leq 10^{-3}$.

$$
\begin{aligned}
4x_1 + x_2 -x_3 + x_4 &= -3,\\
x_1 + 4x_2 -x_3 -x_4 &= -2,\\
-x_1 -x_2 + 5x_3 + x_4 &=5,\\
x_1 -x_2 + x_3 + 3x_4 &=2.\\
\end{aligned}
$$

In [19]:
A = [
    4  1 -1  1;
    1  4 -1 -1;
   -1 -1  5  1;
    1 -1  1  3
];
b = [-3;-2;5;2];
x₀ = [0;0;1;0];

### (1) Gauss-Seidel method

In [20]:
function GaussSeidel(A,b,x₀;ϵ=1e-3,maxiter=1e4,norm=maximum)
    D,L,U=DLU(A)
    invDL = inv(D - L)
    T = invDL*U
    c = invDL*b
    for k in 1:maxiter
        x₁ = T*x₀ + c
        if norm(x₁-x₀) ≤ ϵ
            return x₁
        end
        x₀ = x₁
    end   
end

GaussSeidel (generic function with 1 method)

In [21]:
@latex xᵍ = GaussSeidel(A,b,x₀)

<center>$xᵍ=\begin{pmatrix}-0.7527255627893519\\0.040855425708911985\\0.7195080258246528\\0.691357654224537\end{pmatrix}$</center>


### (2) the SOR method

In [22]:
function SOR(A,b,x₀;ω=1.2,ϵ=1e-3,maxiter=1e4,norm=maximum)
    D,L,U=DLU(A)
    invDL = inv(D - ω*L)
    T = invDL*((1-ω)*D+ω*U)
    c = invDL*ω*b
    for k in 1:maxiter
        x₁ = T*x₀ + c
        if norm(x₁-x₀) ≤ ϵ
            return x₁
        end
        x₀ = x₁
    end   
end

SOR (generic function with 1 method)

In [23]:
@latex xˢ = SOR(A,b,x₀;ω=1.2)

<center>$xˢ=\begin{pmatrix}-0.7530897093888335\\0.04117280965301612\\0.7192518346947181\\0.6916350607145069\end{pmatrix}$</center>
